In [2]:
import altair as alt
import pandas as pd

In [3]:
df = pd.read_csv('https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/licenses_fall2022.csv', parse_dates=['Original Issue Date','Effective Date','Expiration Date'])

In [4]:
df.columns

Index(['_id', 'License Type', 'Description', 'License Number',
       'License Status', 'Business', 'Title', 'First Name', 'Middle',
       'Last Name', 'Prefix', 'Suffix', 'Business Name', 'BusinessDBA',
       'Original Issue Date', 'Effective Date', 'Expiration Date', 'City',
       'State', 'Zip', 'County', 'Specialty/Qualifier',
       'Controlled Substance Schedule',
       'Delegated Controlled Substance Schedule', 'Ever Disciplined',
       'LastModifiedDate', 'Case Number', 'Action', 'Discipline Start Date',
       'Discipline End Date', 'Discipline Reason'],
      dtype='object')

In [5]:
df['Ever Disciplined'].unique()

array(['N', 'Y'], dtype=object)

In [6]:
df['License Status'].unique()

array(['NOT RENEWED', 'ACTIVE', 'CANCELLED', 'TERMINATED CARD RETURNED',
       'DECEASED', 'INACTIVE', 'EXPIRED', 'CLOSED',
       'TERMINATED WITHOUT CARD', 'INOPERATIVE',
       'TERMINATED VALID REASON', 'CHANGE OF OWNERSHIP',
       'Non Sufficient Fund Check Terminated'], dtype=object)

In [7]:
df['License Type'].unique()

array(['DETECTIVE BOARD', 'COSMO', 'FUNERAL AND EMBALMER', 'DENTAL',
       'DESIGN FIRM', 'LIMITED LIABILITY CO', 'ATHLETIC TRAINER',
       'DIETETIC AND NUTRITION', 'APPRAISAL', 'MASSAGE LICENSING BD',
       'ARCHITECT', 'ATHLETICS', 'MAR AND FAM THERAPIST', 'GEOLOGY',
       'HME AND SERVICES PROV', 'COLLECTION AGENCY',
       'ENVIRON. HLTH PRACT', 'HOME INSPECTOR', 'LAND SURVEYOR BOARD',
       'CLIN PSYCHOLOGIST', 'INTERIOR DESIGN', 'LANDSCAPE ARCHITECT',
       'COMM ASSOC MGR', 'IDPR', 'MEDICAL BOARD', 'DETECT. DECEPTION',
       'AUCTIONEER', 'CEMETERY OVERSIGHT'], dtype=object)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 31 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   _id                                      10000 non-null  int64         
 1   License Type                             10000 non-null  object        
 2   Description                              10000 non-null  object        
 3   License Number                           9940 non-null   object        
 4   License Status                           10000 non-null  object        
 5   Business                                 10000 non-null  object        
 6   Title                                    110 non-null    object        
 7   First Name                               9605 non-null   object        
 8   Middle                                   3622 non-null   object        
 9   Last Name                               

In [9]:
df.describe()

,_id,Original Issue Date,Effective Date,Delegated Controlled Substance Schedule,Case Number
count,1.000000e+04,9995,9208,0.0,3.430000e+02
mean,7.787635e+05,1999-11-15 00:17:17.318659328,2005-07-16 12:59:34.978279808,NaN,2.005356e+09
min,2.792630e+05,1912-07-13 00:00:00,1912-07-13 00:00:00,NaN,1.982000e+09
25%,5.264690e+05,1992-03-23 00:00:00,1994-10-13 00:00:00,NaN,1.997526e+09
50%,7.806600e+05,1999-08-26 00:00:00,2010-11-30 12:00:00,NaN,2.004005e+09
75%,1.028364e+06,2011-06-14 00:00:00,2019-08-16 00:00:00,NaN,2.013504e+09
max,1.279042e+06,2022-07-14 00:00:00,2022-07-15 00:00:00,NaN,2.021011e+09
std,2.901112e+05,NaN,NaN,NaN,9.249388e+06


In [10]:
df['Issue Year'] = df['Original Issue Date'].dt.year.astype('Int16')
df = df[(df['Issue Year'] > 1999)&(df['Issue Year'] < 2022)].copy()

In [11]:
licenses_by_year = df.groupby('Issue Year').size().reset_index(name='Number of Licenses')

In [12]:
year_sel = alt.selection_point(name='selYear', value= 2019, fields=['Issue Year'], on='click',nearest=True)

In [13]:
chart1 = alt.Chart(licenses_by_year).mark_bar().encode(
    x=alt.X('Issue Year:O', title='Year'),
    y=alt.Y('Number of Licenses:Q', title='# of Licenses Issued'),
    color=alt.condition(year_sel, alt.value('steelblue'), alt.value('lightblue')),
    tooltip=['Issue Year:O', 'Number of Licenses:Q']).add_params(year_sel).properties(title='Number of Professional Licenses Issued (2000-2021)')
chart1.display()

alt.Chart(...)

In [14]:
# df_year = df[df['Issue Year'] == 2019]
# license_types = df_year['License Type'].value_counts().head(4).index.tolist()
# df_year['License Type'] = df_year['License Type'].apply(lambda x: x if x in license_types else 'Other')
# df_year['License Type'].value_counts()
# df_license = df_year.groupby('License Type').size().reset_index(name='Number of Licenses')
# df_license['Percentage'] = ((df_license['Number of Licenses']/df_license['Number of Licenses'].sum())*100).round(1)
# df_license


In [15]:
# chart2 = alt.Chart(df_license).mark_bar().encode(
#     x=alt.X('Percentage:Q', title=''),
#     y=alt.Y('License Type:N', sort='-x', title='Number of Licenses Issued')
#     ).properties(title='Type of Licenses Issued')
# chart2.display()

In [14]:
df_year_type = df.groupby(['Issue Year', 'License Type']).size().reset_index(name='Count')
df_year_type['YearTotal'] = df_year_type.groupby('Issue Year')['Count'].transform('sum')
df_year_type['Percentage'] = (df_year_type['Count'] / df_year_type['YearTotal']).round(2)

In [15]:
chart2 = (alt.Chart(df_year_type).mark_bar().encode(
    x=alt.X('Percentage:Q').axis(format='%'),
    y=alt.Y('License Type:N', sort='-x', title='License Type'),
    tooltip=['Issue Year:O', 'License Type:N', 'Percentage:Q']).transform_filter(year_sel).properties(title='License Type Distribution (Selected Year)'))
chart2.display()

alt.Chart(...)

In [16]:
(chart1 | chart2)

alt.HConcatChart(...)

In [20]:
!pwd

/workspaces/manuelarodr.github.io/python_notebooks


In [17]:
charts = (chart1 | chart2)
charts.save("../assets/json/licenses_plots.json")